<h1><b>Markov Decision Processes</h1></b>
<p align="justify">Στη συγκεκριμένη άσκηση θα μελετήσετε τους αλγορίθμους <i>Policy Iteration</i> και <i>Value Iteration</i>, καθώς και θα εξοικειωθείτε με βασικές έννοιες των <i>Markov Decision Processes</i>. Οι αλγόριθμοι <i>Policy Iteration</i> και <i>Value Iteration</i> είναι από τους βασικούς αλγορίθμους δυναμικού προγραμματισμού που χρησιμοποιούνται για την επίλυση της εξίσωσης <i>Bellman</i> σε <i>Markov Decision Processes</i>.</p> 
<p align="justify">Το πρόβλημα που θα μελετήσετε είναι αυτό της παγωμένης λίμνης (Frozen Lake) με μέγεθος πλέγματος 8 x 8.</p>


<h2><b>Εξοικείωση με τη βιβλιοθήκη <i>Gym</i></b></h2>

In [1]:
import numpy as np
import gym
from gym import wrappers

Με την παρακάτω εντολή, ορίζετε το πρόβλημα που θα μελετηθεί:

In [2]:
env_name = 'FrozenLake8x8-v0'
env = gym.make(env_name)

Με τις παρακάτω εντολές, θα επαναφέρετε τον Agent στην αρχική του θέση και θα οπτικοποιήσετε το πλέγμα και τη θέση του Agent

In [15]:
env.reset()
env.render()


SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


Με τις παρακάτω εντολές, ορίζετε την επόμενη ενέργεια με τυχαίο τρόπο και ο Agent κάνει ένα βήμα.

In [28]:
next_action = env.action_space.sample()
#print(next_action) # 0 left , 1 down , 2 right, 3 up      
env.step(next_action)
env.render()

  (Left)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG


In [18]:
next_action = env.action_space.sample()
env.step(next_action ) # probability 0.33 , # False is if move was done or not

(8, 0.0, False, {'prob': 0.3333333333333333})

__Ερώτηση__

Να εκτελέσετε αρκετές φορές τις τελευταίες εντολές και να παρατηρήσετε κάθε φορά την ενέργεια που ζητείται από τον agent να εκτελέσει και την ενέργεια που αυτός πραγματοποιεί. Πραγματοποιεί πάντοτε ο agent την κίνηση που του ζητείται; Είναι ντετερμινιστικές ή στοχαστικές οι κινήσεις του agent;


__Απάντηση__

Παρατηρούμε πως ο Agent δεν πάει σε όποια κατεύθυνση του δηλώσουμε. Επομένως οι κινήσεις του δεν είναι ντετερμινιστικές αλλά στοχαστικές. Συγκεκριμένα η εντολή env.step κάνει το βήμα με πιθανότητα 1/3.


<h2><b>Ερωτήσεις</b></h2>

__Ερώτηση__ 

Μελετώντας <a href="https://gym.openai.com/envs/FrozenLake-v0/">αυτό</a> και βασισμένοι στα συμπεράσματα του προηγούμενου ερωτήματος να περιγράψετε σύντομα το πρόβλημα της παγωμένης λίμνης ως πρόβλημα βελτιστοποίησης. Ποιος είναι ο στόχος του agent;
    
__Aπάντηση__    
     
   Στο συγκεκριμένο πρόβλημα ο agent ελέγχει τις κινήσεις ενός χαρακτήρα μέσα σε ένα πλέγμα 8χ8 διαστάσεων. Τα κελιά περιέχουν τις τιμές F(frozen),H(Hole),G(final goal), S(start cell). Κάποια κελιά είναι αδύνατον να περπατηθούν όπως τα H γιατί ο χαρακτήρας μας θα πέσει μέσα. Ο agent μπορεί να κινηθεί μόνο σε F κελιά επειδή είναι παγωμένα. Ο agent μπορεί να κάνει οριζόντιες ή κάθετες κινήσεις στο πλέγμα αλλά όχι διαγώνιες. Ξεκινάει από το κελί S και έχει σαν στόχο να φτάσει στο κελί G. O agent βραβεύεται όταν βρίσκει ένα καλό path για το G κελί. Eπομένως ο στόχος του είναι να βρει το optimal policy, δηλαδή ποια κατεύθυνση να διαλέξει σε κάθε κατάσταση για να φτάσει στο επιθυμητό αποτέλεσμα για να μεγιστοποιήσει τα rewards. Να σημειωθεί πως οι κινήσεις του agent είναι στοχαστικές και όχι ντετερμινιστικές, δηλαδή έχει πιθανότητα 1/3 να πάει εκεί που θέλει,καθώς τα κελιά "γλιστράνε" επομένως αλλάζει κατεύθυνση.


__Ερώτηση__

Να διατυπώσετε την ιδιότητα <i>Markov</i>. Πώς απλοποιεί η ιδιότητα αυτή τη μελέτη του συγκεκριμένου προβλήματος;

__Aπάντηση__  

Μια στοχαστική διαδικασία έχει την ιδιότητα Markov εάν η δεσμευμένη πιθανοτήτα κατανομής των μελλοντικών καταστάσεων της διαδικασίας εξαρτάται μόνο από την παρούσα κατάσταση και όχι από την ακολουθία των γεγονότων που προηγήθηκαν.

<img src="mp.png" />

Επομένως η παραπάνω ιδιότητα μας απλοποιεί το πρόβλημα καθώς κοιτάμε μόνο την κατάσταση που βρισκόμαστε και όχι όλες τις προηγούμενες(memoryless system).

Eπομένως τα transition models ή reward models έχουν την παρακάτω μορφή:

<img src="models.png" /> 
όπου a -> actions in A(set of actions) και s -> states in S (set of states).

Το Α λαμβάνει τις τιμές {0,1,2,3}.

Το S λαμβάνει τις τιμές {0,1,2...,63}

Σε άλλη περίπτωση θα είχαμε ένα πολύπλοκο μοντέλο το οποίο θα ήταν δύσκολο να υπολογιστεί.

__Ερώτηση__

Να περιγράψετε σύντομα τους αλγορίθμους <i>Policy Iteration</i> και <i>Value Iteration</i>, δίνοντας έμφαση στο διαφορετικό τρόπο με τον οποίο προσεγγίζουν την επίλυση του προβλήματος. Είναι εγγυημένο ότι οι δύο αλγόριθμοι θα συγκλίνουν στη βέλτιστη πολιτική; Αν ναι, οδηγούν σε ίδια ή διαφορετική βέλτιστη πολιτική;

__Aπάντηση__  

Πριν την απάντηση καλό θα ήταν να ορίσουμε κάποιες παραμέτρους.

<h6><i>Policy</i></h6> είναι ο τρόπος με τον οποίο ένας agent διαλέγει ένα action. Γενικά η συνάρτηση policy δέχεται καταστάσεις και επιστρέφει actions. Στο πρόβλημα μας το policy είναι μεγέθους 64 και αποτελείται απο actions για κάθε state π.χ. π = [0,1,3,....2]

<img src="policy.png" />

<h6><i>Total Reward</i></h6> είναι οι ανταμοιβές r μέχρι ένα timestep T (r_1 + r_2+...r_T). Όπου τα reward έχουν τιμές 1 για μετάβαση σε GOAL state και 0 αλλιώς. Tο total discounted  award καθορίζεται από τον παρακάτω τύπο όπου όσο πιο μεγάλο γ έχουμε σημαίνει πως μας ενδιαφέρουν τα μελλοντικά rewards πάρα πολύ , ενώ όσο πιο χαμηλό είναι μας ενδιαφέρουν τα immediate rewards. Στην περίπτωση μας έχουμε θέση γ=1.0 , οπότε δε χρειάζεται στον τύπο

<img src="tdr.png" /> 

<h6><i>Transition Probabilities</i></h6> επιπλέον ορίζουμε τις πιθανότητες μετάβασης ως εξής. Για ένα next state S_next δεδομένου ενός State S και ενός action a έχουμε : P(S_nexT| S,a) = 1/3. Όμως μπορεί να πάει και σε άλλες 2 καταστάσεις με πιθανότητα 1/3 για παράδειγμα, δηλαδή αν η κίνηση που θέλουμε είναι να πάμε δεξιά, τότε: P(s_right|s,a) = P(s_left|s,a) = P(s_down|s,a) = 1/3. Βλέπουμε πως το πρόγραμμα αφαιρεί μια κατάσταση ώστε να αθροίζουν οι πιθανότητες στο 1. Αυτό φαίνεται από την εντολή __env.P[s][a]__ που επιστρέφει 3 tuples ένα για κάθε πιθανό state που μπορεί να πάει από την S, τα οποία tuples περιέχουν την πιθανότητα μετάβασης στην νέα κατάσταση, τη νέα κατάσταση , το reward και μια boolean τιμή για το αν έγινε η κίνηση ή όχι(done).

π.χ. print(env.P[62][3]) 

outputs: [(0.3333333333333333, 63, 1.0, True), (0.3333333333333333, 54, 0.0, True), (0.3333333333333333, 61, 0.0, False)]
    


<h6><i>Value Function</i></h6> Δείχνει πόσο καλή είναι μια κατάσταση για έναν agent ώστε να είναι εκεί. Αλλιώς ορίζεται ως το Expected Total Reward για κάθε στοιχείο αν βρίσκεται στην κατάσταση S. Έπειτα από πράξεις καταλήγουμε:

<img src="bellmaneq.png" /> , όπου R(s,a) είναι το immediate reward και το άλλο σκέλος είναι το discounted expected reward πηγάινοντας στο s'.

Η παραπάνω συνάρτηση μεγιστοποιείται βρίσκοντας το βέλτιστο policy __π*__  δηλαδή τα __α__ (actions) που θα φέρουν το μέγιστο κέρδος.

<h6><i>Q function</i></h6>Τέλος να αναφέρουμε τη συνάρτηση __Q:SXA -> R__ , δηλαδή λαμβάνει ένα action και το state που είναι και επιστρέφει μια πραγματική τιμή. Aυτή η πραγματική τιμή συμβολίζει το expected total reward όταν είσαι στη κατάσταση __S__ και διαλέγεις action __α__. Επομένως η V ορίζεται και με τους παρακάτω τρόπους

__V*(s) = max{Q*(s,a)}__, over a, for every s    ή     __π* = arg max {Q*(s,a)}__, over a, for every s

 
<h3>Value iteration:</h3>Υπολογίζει την βέλτιστη κατάσταση για τη value function, βελτιώνοντας επαναληπτικά την εκτίμηση της στην κατάσταση S (V(s)). Ενημερώνει επανειλημμένα τις τιμές Q (s, a) και V (s) έως ότου συγκλίνουν. Ο αλγόριθμος αυτός είναι σίγουρο ότι συγκλίνει σε βέλτιστες τιμές.

παρακάτω φαίνεται ο ψευδοκώδικας.
<img src="vi.png" />

Βλέπουμε πως πρώτο βήμα έχει την αρχικοποιήση τυχαία των τιμών της __V(s)__. Έπειτα, προσπαθεί να μεγιστοποιήσει τη συνάρτηση __Q__ επαναλαμβάνοντας τον εξής υπολογισμό για έναν ορισμένο αριθμό επαναλήψεων.
Να βρει δηλαδή για κάθε state s το α το οποίο μεγιστοποιεί την __Q__. Ουσιαστικά, βρίσκει το __V*__ κάνοντας διερεύνηση(exploration) όλο το χώρο καταστάσεων και ενεργείων. Έπειτα έχοντας το __optimal V(V*)__ μπορεί να κάνει extract το αντίστοιχο policy κάνοντας χρήση της μεθόδου __extract policy__. Η οποία μέθοδος ουσιαστικά επιλύει την εξισωση : __π* = arg max {Q*(s,a)}, over a, for every s__.


<h3>policy iteration:</h3> Ο συγκεκριμένος αλγόριθμος αντί να βελτιώνει την εκτίμηση της value function, αναπροσδιορίζει το policy σε κάθε βήμα και υπολογίζει σύμφωνα με αυτό τις τιμές v(s) μέχρις ότου να συγκλίνει  σε ένα π. Και αυτός ο αλγόριθμος είναι σίγουρο ότι συγκλίνει σε βέλτιστες τιμές.

παρακάτω φαίνεται ο ψευδοκώδικας,όπου E[r|s,a] το immediate reward (R(s,a)).
<img src="pi.png" /> 

Παρατηρούμε πως πρώτο βήμα είναι η αρχικοποιήσει ενός policy(μια ακολουθία π με 64 actions ένα για κάθε κατάσταση) με τυχαίο τρόπο.Αυτό το κάνει η συνάρτηση __policy iteration__. Έπειτα η συνάρτηση αυτή καλεί την __compute_policy_v__ η οποία υπολογίζει το __V(s)__ για κάθε state λύνοντας αυτήν την εξίσωση:

__V*(s) = max{Vπ(s)}__, over π, for every s ή __Vπ(s) = Σ(1/3)*(r_i + Vπ(s'))__ , for all s', over a, for every s.

Στη συνέχεια καλείται η __extract policy__ μέθοδος που βρίσκεται μέσα στην __policy iteration__. Aυτή η συνάρτηση ουσιαστικά επιλύει την εξίσωση __π* = arg max {Vπ(s)}__, over π, for every s.

Οι παραπάνω διαδικασίες __compute_policy_v__ και __extract policy__ τρέχουν ουσιαστικά εναλλάξ για ένα αριθμό max_iterations, εώς ότου συγκλίνουν τα old_π με τα new_π. (if (np.all(policy == new_policy))).

<h6>Πως διαφέρουν οι δύο αλγόριθμοι </h6>
<i>η διαφορά των 2 αλγορίθμων είναι πως ο ένας προσπαθεί να μεγιστοποιήσει το V(s) και στη συνέχεια από αυτό να εξάγει τo βέλτιστο policy. Ενώ ο policy iteration αλγόριθμος προσπαθεί αμέσως να βρει το βέλτιστο π, χωρίς να βρει το βέλτιστο V(s). Ουσιαστικά πετυχάινουν τον ίδιο σκοπό με άλλο τρόπο. Ο value iteration ανγακάζεται ουσιαστικά να κάνει ένα παραπάνω βήμα από τον policy iteration.</i>

<h6> Oδηγούν σε ίδια ή διαφορετική βέλτιστη πολιτική; </h6>
Oι δύο αλγόριθμοι βρίσκουν βέλτιστες τιμές, ωστόσο ο αλγόριθμος policy iteration συγκλίνει ταχύτερα, λόγω του ότι κάνει λιγότερες επαναλήψεις. Ωστόσο κάθε επανάληψη του είναι πιο αργή-περίπλοκη από του value iteration.

__Ερώτηση__

Να εκτελέσετε τα προγράμματα που σας δίνονται, τα οποία επιλύουν το
πρόβλημα της παγωμένης λίμνης, χρησιμοποιώντας τους αλγορίθμους <i>Policy
Iteration</i> και <i>Value Iteration</i> αντίστοιχα. Ποια μέθοδος συγκλίνει στη βέλτιστη λύση σε λιγότερα βήματα; Τι συμπέρασμα βγάζετε; Να ελέγξετε το χρόνο εκτέλεσης του κάθε προγράμματος, χρησιμοποιώντας την εντολή <i>time</i>. Τι συμπέρασμα βγάζετε ως προς την πολυπλοκότητα του κάθε αλγορίθμου;

__Aπάντηση__  

Η μέθοδος policy iteration συγκλίνει σε μόλις 13 βήματα ενώ η value iteration σε 2357. Επομένως, παρατηρούμε πως ο 1ος αλγόριθμος είναι καλύτερος ως προς τον αριθμό των βημάτων.

Ωστόσο η μέθοδος value iteration ήθελε λιγότερα δευτερόλεπτα για να συγκλίνει. Γεγονός που σημαίνει ότι μπορεί να κάνει πολλές επαναλήψεις αλλά είναι πάρα πολύ γρήγορες, ενώ η μέθοδος policy iteration κάνει λίγες επαναλήψεις αλλά αργές. Οι δύο αλγόριθμοι σύγκλιναν στο ίδιο average score = 1.0.

Είναι εμφανές πως ο αλγόριθμος policy iteration είναι αρκετά πιο πολύπλοκος στη λύση του από τον value-iteration

<h2><b>Policy Iteration</b></h2>

In [4]:
"""
Solving FrozenLake8x8 environment π* = arg max {Q*(s,a)}, over a, for every s using Policy iteration.
Author : Moustafa Alzantot (malzantot@ucla.edu)
"""
import numpy as np
import gym
from gym import wrappers
import time

def run_episode(env, policy, gamma = 1.0, render = False):
    """ Runs an episode and return the total reward """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0, n = 100):
    scores = [run_episode(env, policy, gamma, False) for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.nA)
        for a in range(env.nA):
            q_sa[a] = sum([p * (r + gamma * v[s_]) for p, s_, r, _ in  env.P[s][a]])
        policy[s] = np.argmax(q_sa)
    return policy

def compute_policy_v(env, policy, gamma=1.0):
    """ Iteratively evaluate the value-function under policy.
    Alternatively, we could formulate a set of linear equations in iterms of v[s] 
    and solve them to find the value function.
    """
    v = np.zeros(env.nS)
    eps = 1e-10
    while True:
        prev_v = np.copy(v)
        for s in range(env.nS):
            policy_a = policy[s]
            v[s] = sum([p * (r + gamma * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if (np.sum((np.fabs(prev_v - v))) <= eps):
            # value converged
            break
    return v

def policy_iteration(env, gamma = 1.0):
    """ Policy-Iteration algorithm """
    policy = np.random.choice(env.nA, size=(env.nS))  # initialize a random policy , 64 shape with 0-3 values for up,down,left,right
    max_iterations = 200000
    gamma = 1.0
    for i in range(max_iterations):
        old_policy_v = compute_policy_v(env, policy, gamma)
        #print(policy) 
        #print(old_policy_v,old_policy_v.shape) V[s] values
        new_policy = extract_policy(old_policy_v, gamma) # policy - moves to do
        #print(new_policy,new_policy.shape)# 64 array, each cell with 0 to 3 for each kind of move]
        #print(new_policy)
        if (np.all(policy == new_policy)):# the initial policy converged to new policy
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        policy = new_policy
    return policy


if __name__ == '__main__':
    env_name  = 'FrozenLake8x8-v0'
    env = gym.make(env_name)
    env = env.unwrapped# https://stackoverflow.com/questions/53836136/why-unwrap-an-openai-gym
    start_time = time.time()
    optimal_policy = policy_iteration(env, gamma = 1.0)
    scores = evaluate_policy(env, optimal_policy, gamma = 1.0)
    print("--- %s seconds ---" % (time.time() - start_time))
    print('Average scores = ', np.mean(scores))

Policy-Iteration converged at step 12.
--- 2.924541473388672 seconds ---
Average scores =  1.0


In [7]:
#policy = np.random.choice(env.nA, size=(env.nS))
#policy

array([1, 0, 2, 3, 3, 0, 1, 3, 1, 2, 3, 2, 1, 0, 3, 2, 1, 0, 0, 2, 0, 3,
       2, 2, 3, 1, 1, 0, 2, 3, 2, 2, 2, 1, 2, 1, 1, 1, 3, 1, 2, 0, 1, 0,
       2, 1, 2, 1, 3, 2, 3, 1, 1, 0, 1, 1, 3, 0, 3, 3, 3, 2, 2, 1])

In [11]:
# state 2 - action 0
print(env.P[62][3])
print("----------------")
for p, s_, r, underscore in env.P[62][3] :
    print(p) # probability
    print(s_) # next_state
    print(r) # reward
    print( underscore)
    print("----------------")
# Reward is 1 when next state is Final and, 0 otherwise

[(0.3333333333333333, 63, 1.0, True), (0.3333333333333333, 54, 0.0, True), (0.3333333333333333, 61, 0.0, False)]
----------------
0.3333333333333333
63
1.0
True
----------------
0.3333333333333333
54
0.0
True
----------------
0.3333333333333333
61
0.0
False
----------------


<h2><b>Value Iteration</b></h2>

In [15]:
"""
Solving FrozenLake8x8 environment using Value-Itertion.
Author : Moustafa Alzantot (malzantot@ucla.edu)
"""
import numpy as np
import gym
from gym import wrappers
import time

def run_episode(env, policy, gamma = 1.0, render = False):
    """ Evaluates policy by using it to run an episode and finding its
    total reward.
    args:
    env: gym environment.
    policy: the policy to be used.
    gamma: discount factor.
    render: boolean to turn rendering on/off.
    returns:
    total reward: real value of the total reward recieved by agent under policy.
    """
    obs = env.reset()
    total_reward = 0
    step_idx = 0
    while True:
        if render:
            env.render()
        obs, reward, done , _ = env.step(int(policy[obs]))
        total_reward += (gamma ** step_idx * reward)
        step_idx += 1
        if done:
            break
    return total_reward


def evaluate_policy(env, policy, gamma = 1.0,  n = 100):
    """ Evaluates a policy by running it n times.
    returns:
    average total reward
    """
    scores = [
            run_episode(env, policy, gamma = gamma, render = False)
            for _ in range(n)]
    return np.mean(scores)

def extract_policy(v, gamma = 1.0):
    """ Extract the policy given a value-function """
    policy = np.zeros(env.nS)
    for s in range(env.nS):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + gamma * v[s_]))
        policy[s] = np.argmax(q_sa)
    return policy


def value_iteration(env, gamma = 1.0):
    """ Value-iteration algorithm """
    v = np.zeros(env.nS)  # initialize value-function
    max_iterations = 100000
    eps = 1e-20
    for i in range(max_iterations):
        prev_v = np.copy(v)
        for s in range(env.nS):
            q_sa = [sum([p*(r + prev_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.nA)] 
            v[s] = max(q_sa)
        if (np.sum(np.fabs(prev_v - v)) <= eps):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    return v


if __name__ == '__main__':
    env_name  = 'FrozenLake8x8-v0'
    gamma = 1.0
    env = gym.make(env_name)
    env = env.unwrapped
    start_time = time.time()
    optimal_v = value_iteration(env, gamma);
    policy = extract_policy(optimal_v, gamma)
    policy_score = evaluate_policy(env, policy, gamma, n=1000)
    print("--- %s seconds ---" % (time.time() - start_time))
    print('Policy average score = ', policy_score)

Value-iteration converged at iteration# 2357.
--- 2.626422882080078 seconds ---
Policy average score =  1.0
